In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Loads key generated in Notebook1, key must be in dictionary in file 'tokens.json'
with open('tokens.json') as f:
    tokens = json.load(f)

In [ ]:
#Retrieves key from json file
key = tokens['key']['key']

In [ ]:
waypoints = pd.read_csv('waypoints.csv')

In [ ]:
#strips extra whitespace off end of every entry
waypoints['STREET1'] = waypoints['STREET1'].str.strip()
waypoints['CITY'] = waypoints['CITY'].str.strip()
waypoints['STATE'] = waypoints['STATE'].str.strip()

In [ ]:
waypoints.head() #check data, must have columns from above cells

In [ ]:
key #must have Bing Maps Key to use a Bing Maps API

In [ ]:
#unchanging parts of each url used for requests to the 
base = 'http://dev.virtualearth.net/REST/V1/Routes/Driving?'
waypoint0 = 'wp.0='
waypoint1 = '&wp.1='
tail = '&optmz=distance&output=json&key='
endpoint = 'Chicago, IL'
endpoint = endpoint.replace(' ','%')
endpoint = endpoint.replace(',','')

In [ ]:
#Creates lists of all street addresses, cities, states in order
street1_list = list(waypoints.STREET1)
city_list = list(waypoints.CITY)
state_list = list(waypoints.STATE)

In [ ]:
#Gets json file returned from Bing Maps Driving Distance Matrix API for each address is waypoints
#Prints out all 400 responses (Bad Request) meaning address not found
#Returns the response code and the json text
def API_func(address,url,j):
    response = requests.get(url)
    if response.reason == 'Bad Request':
        print(j,response, response.reason, address)
        j = j + 1
    else:
        return response, response.text, j
    return response, response.text, j

In [ ]:
#Parses json file and pulls total drive distance
def distance_func(text):
    parsed = json.loads(text)
    parsed = parsed['resourceSets'][0]
    parsed = parsed['resources'][0]
    distance = parsed['travelDistance']
    return(distance)

In [ ]:
#for loop iterates through each row of waypoints, assigns variables, builds request urls, appends distance list
#Calls appropriate functions
distances = list()
j = 0
for i in range(len(waypoints)):
    street1 = str(street1_list[i])
    city = str(city_list[i])
    state = str(state_list[i])
    address = street1 + ' ' + city + ' ' + state
    address = address.replace(' ','%')
    url = base + waypoint0 + address + waypoint1 + endpoint + tail + key
    #print(url)
    response, text, j = API_func(address,url,j)
    if response.reason != 'OK':
        distances.append('')
        #print(address)
    else:
        distance = distance_func(text)
        distances.append(distance)
waypoints['distance'] = distances

In [ ]:
#Adds driving distance (in km) to waypoints
waypoints['distance'] = distances

In [ ]:
#removes points not found from waypoints
#converts distances from km to miles
waypoints_plot = waypoints[waypoints['distance'] != '']
waypoints_plot.distance = waypoints_plot.distance/1.609

In [ ]:
#exports data
waypoints.to_csv('waypoints.csv')

In [ ]:
#last json file printed for viewing
parsed = json.loads(response.text)
#parsed = parsed['resourceSets'][0]
print(json.dumps(parsed, indent=4, sort_keys=True))

In [ ]:
#example plot (histogram)
waypoints_plot[["distance"]] = waypoints_plot[["distance"]].apply(pd.to_numeric)
waypoints_plot
plt.hist(x=waypoints_plot['distance'],bins=50)
plt.xlabel('One Way Drive Distance (Miles)')
plt.ylabel('Count')
plt.title('Drive Distance Distribution')
plt.legend()
plt.show()